Regresión logística con bajo accuracy

In [12]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Cargar los datos
data = pd.read_csv("appliance_usage_data2.csv")

# Convertir la columna "Hora" a cadena (string)
data["Hora"] = data["Hora"].astype(str)

# Verificar y corregir el formato de la columna "Hora"
data["Hora"] = data["Hora"].apply(lambda x: x.zfill(2) if len(x) == 1 else x)

# Combinar las columnas "Fecha" y "Hora" en una sola columna datetime
data["Fecha_Hora"] = pd.to_datetime(data["Fecha"] + " " + data["Hora"], format="%Y-%m-%d %H:%M:%S", errors='coerce')

# Eliminar las filas con fechas incorrectas
data = data.dropna(subset=["Fecha_Hora"])

# Codificar la columna "Reporte de fallo" a valores numéricos
data["Reporte de fallo"] = data["Reporte de fallo"].map({0: 0, 1: 1})

# Crear características a partir de la columna "Fecha_Hora"
data["Hora_del_dia"] = data["Fecha_Hora"].dt.hour
data["Dia_de_la_semana"] = data["Fecha_Hora"].dt.dayofweek
data["Mes"] = data["Fecha_Hora"].dt.month

# Eliminar las columnas "Fecha" y "Hora" originales
data = data.drop(["Fecha", "Hora", "Fecha_Hora"], axis=1)

# Separar las variables de entrada y salida
X = data[["Hora_del_dia", "Watts usados", "Porcentaje de fallos", "Dia_de_la_semana", "Mes"]]
y = data["Reporte de fallo"]

# Dividir los datos en un conjunto de entrenamiento y un conjunto de prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Entrenar el modelo
model = LogisticRegression()
model.fit(X_train, y_train)

# Predecir los datos de prueba
y_pred = model.predict(X_test)

# Evaluar el rendimiento del modelo
print(f"Accuracy: {accuracy_score(y_test, y_pred):.2f}")

ValueError: With n_samples=0, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

Clasificación binaria con ruido

In [34]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Paso 1: Cargar y preparar los datos
data = pd.read_csv('appliance_usage_data2.csv')
X = data[['Fecha', 'Hora', 'Watts usados']]
y = data['Reporte de fallo']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Paso 2: Preprocesamiento de datos
X_train['Fecha'] = pd.to_datetime(X_train['Fecha'])
X_train['DiaDelAño'] = X_train['Fecha'].dt.dayofyear
X_train = X_train.drop(columns=['Fecha'])
scaler = StandardScaler()
X_train[['Hora', 'DiaDelAño']] = scaler.fit_transform(X_train[['Hora', 'DiaDelAño']])

# Paso 3: Crear y entrenar el modelo de clasificación
model = keras.Sequential([
    layers.Input(shape=(3,)),
    layers.Dense(128, activation='relu'),
    #layers.Dropout(0.2),  # Agregando ruido para evitar el sobreajuste
    layers.Dense(64, activation='relu'),
    #layers.Dropout(0.2),  # Agregando ruido para evitar el sobreajuste
    layers.Dense(32, activation='relu'),
    #layers.Dropout(0.2),  # Agregando ruido para evitar el sobreajuste
    layers.Dense(1, activation='sigmoid') 
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=5, batch_size=32)

# Paso 4: Evaluar el modelo
X_test['Fecha'] = pd.to_datetime(X_test['Fecha'])
X_test['DiaDelAño'] = X_test['Fecha'].dt.dayofyear
X_test = X_test.drop(columns=['Fecha'])
X_test[['Hora', 'DiaDelAño']] = scaler.transform(X_test[['Hora', 'DiaDelAño']])

loss, accuracy = model.evaluate(X_test, y_test)
print(f'Pérdida en el conjunto de prueba: {loss}')
print(f'Precisión en el conjunto de prueba: {accuracy}')

Epoch 1/5
 3810/12500 [========>.....................] - ETA: 15s - loss: 0.7052 - accuracy: 0.4999

KeyboardInterrupt: 

CNN

In [83]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Carga tus datos desde un archivo CSV (asegúrate de que tu archivo de datos esté en el mismo directorio que este script)
data = pd.read_csv('appliance_usage_data2.csv')

# Preprocesamiento de datos
X = data[['Hora', 'Watts usados', 'Porcentaje de fallos', 'Probabilidad de fallo']].values
y = data['Reporte de fallo'].values

# Normaliza los datos
scaler = StandardScaler()
X = scaler.fit_transform(X)

# División de datos en conjuntos de entrenamiento, validación y prueba
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Diseño del modelo
model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(4,)),  # 4 características de entrada
    layers.Dense(1, activation='sigmoid')  # Capa de salida para la clasificación binaria
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Entrenamiento del modelo
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))

# Evaluación del modelo en el conjunto de prueba
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Pérdida en conjunto de prueba: {loss}')
print(f'Precisión en conjunto de prueba: {accuracy}')

# Guardar el modelo en un archivo
model.save("modelo_keras.h5")

Epoch 1/10
10938/10938 [==============================] - 22s 2ms/step - loss: 0.5060 - accuracy: 0.7486 - val_loss: 0.5024 - val_accuracy: 0.7495
Epoch 2/10
10938/10938 [==============================] - 21s 2ms/step - loss: 0.5027 - accuracy: 0.7486 - val_loss: 0.5026 - val_accuracy: 0.7489
Epoch 3/10
10938/10938 [==============================] - 23s 2ms/step - loss: 0.5021 - accuracy: 0.7489 - val_loss: 0.5012 - val_accuracy: 0.7494
Epoch 4/10
10938/10938 [==============================] - 20s 2ms/step - loss: 0.5019 - accuracy: 0.7487 - val_loss: 0.5009 - val_accuracy: 0.7494
Epoch 5/10
10938/10938 [==============================] - 23s 2ms/step - loss: 0.5018 - accuracy: 0.7488 - val_loss: 0.5017 - val_accuracy: 0.7489
Epoch 6/10
10938/10938 [==============================] - 22s 2ms/step - loss: 0.5017 - accuracy: 0.7486 - val_loss: 0.5009 - val_accuracy: 0.7483
Epoch 7/10
10938/10938 [==============================] - 20s 2ms/step - loss: 0.5016 - accuracy: 0.7488 - val_loss: 0

c:\Users\sebas\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Predicciones

In [106]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from keras.models import load_model  # Importa la función load_model de Keras

# Carga tus datos desde un archivo CSV
data = pd.read_csv('nuevos_datos2.csv')

# Preprocesa tus datos de entrada
X = data[['Hora', 'Watts usados', 'Porcentaje de fallos', 'Probabilidad de fallo']].values

# Normaliza los datos
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Carga el modelo Keras desde el archivo h5
model = load_model('modelo_keras.h5')

# Hacer predicciones en nuevos datos
X_new_data = np.array([[6, 0.0, 42.68, 0.5], [18, 200.0, 0.90, 0.6]])
X_new_data = scaler.transform(X_new_data)  # Asegúrate de normalizar también los datos de predicción
predictions = model.predict(X_new_data)

print("Predicciones:", predictions)

# Las predicciones son probabilidades de fallo, puedes establecer un umbral para clasificarlas en 0 o 1 según tu criterio
threshold = 0.5
predicted_labels = (predictions > threshold).astype(int)

print("Predicciones:")
for i in range(len(X_new_data)):
    print(f'Datos: {X_new_data[i]}, Probabilidad de fallo: {predictions[i]}, Clasificación: {predicted_labels[i]}')


1/1 [==============================] - 0s 69ms/step
Predicciones: [[0.48649865]
 [0.5942566 ]]
Predicciones:
Datos: [-0.79502632 -0.19996123  1.00258926  0.0011068 ], Probabilidad de fallo: [0.48649865], Clasificación: [0]
Datos: [ 0.93756927  0.91907683 -1.35962499  0.34766094], Probabilidad de fallo: [0.5942566], Clasificación: [1]


Modelo de XGBoost

In [1]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import joblib  # Importa la biblioteca joblib

# Crear el DataFrame con tus datos
data = pd.read_csv("appliance_usage_data.csv")

# Separar las variables de entrada y salida
X = data[['Hora', 'Watts usados', "Porcentaje de fallos", "Porcentaje de uso", "Probabilidad de fallo"]]
y = data["Reporte de fallo"]

# Dividir los datos en un conjunto de entrenamiento y un conjunto de prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear el modelo de Gradient Boosting (XGBoost)
model = xgb.XGBClassifier(objective="binary:logistic", random_state=42)

# Entrenar el modelo
model.fit(X_train, y_train)

# Guardar el modelo entrenado en un archivo
model_filename = "xgboost_model.pkl"
joblib.dump(model, model_filename)

# Predecir los datos de prueba
y_pred = model.predict(X_test)

# Evaluar el rendimiento del modelo
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

# Imprimir un informe de clasificación
print(classification_report(y_test, y_pred))

Accuracy: 0.75
              precision    recall  f1-score   support

           0       0.75      0.76      0.75     50069
           1       0.75      0.74      0.75     49931

    accuracy                           0.75    100000
   macro avg       0.75      0.75      0.75    100000
weighted avg       0.75      0.75      0.75    100000



Predicciones

In [2]:
import joblib

# Cargar el modelo previamente guardado
model_filename = "xgboost_model.pkl"
loaded_model = joblib.load(model_filename)

# Abriendo un csv con nuevos datos
new_data = pd.read_csv("appliance_usage_data.csv")

# Seleccionando las características de entrada
X_new_data = new_data[['Hora', 'Watts usados', "Porcentaje de fallos", "Porcentaje de uso", "Probabilidad de fallo"]].values

# Hacer predicciones en nuevos datos
predictions = loaded_model.predict(X_new_data)

print("Predicciones:")
print(predictions)

# Guardar las predicciones en un archivo txt.
with open("predicciones3.txt", "w") as output:
    for row in predictions:
        output.write(str(row) + '\n')

Predicciones:
[1 1 0 ... 1 0 1]


In [ ]:
# Importa las bibliotecas necesarias
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Cargando tus datos desde un archivo CSV
data = pd.read_csv('nuevos_datos2.csv')

# Separando las características de entrada (X) y la variable de salida (y)
X = data[['Hora', 'Watts usados', "Porcentaje de fallos", "Probabilidad de fallo"]]
y = data['Reporte de fallo']

# Dividir el conjunto de datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Escalando las características para que tengan media 0 y desviación estándar 1
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Creando un modelo de regresión logística
model = LogisticRegression()

# Entrenando el modelo con los datos de entrenamiento
model.fit(X_train, y_train)

# Realiza predicciones en el conjunto de prueba
y_pred = model.predict(X_test)

# Evaluando el modelo
accuracy = accuracy_score(y_test, y_pred)
print(f'Exactitud del modelo: {accuracy}')

# Enseñando un informe de clasificación
print(classification_report(y_test, y_pred))

Exactitud del modelo: 0.75175
              precision    recall  f1-score   support

           0       0.75      0.75      0.75     50108
           1       0.75      0.75      0.75     49892

    accuracy                           0.75    100000
   macro avg       0.75      0.75      0.75    100000
weighted avg       0.75      0.75      0.75    100000



XGBoost

In [24]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

# Cargar los datos
data = pd.read_csv("appliance_usage_data21.csv")  # Reemplaza "tu_dataset.csv" con la ruta a tu archivo CSV

# Convertir las columnas 'Fecha' y 'Hora' a tipo datetime
data["Fecha"] = pd.to_datetime(data["Fecha"])
data["Hora"] = pd.to_datetime(data["Hora"])

# Extraer características de fecha y hora
data["Día"] = data["Fecha"].dt.day
data["Mes"] = data["Fecha"].dt.month
data["Año"] = data["Fecha"].dt.year
data["Hora del día"] = data["Hora"].dt.hour
data["Día de la semana"] = data["Fecha"].dt.dayofweek

# Codificar el Estado de Componentes usando one-hot encoding
data = pd.get_dummies(data, columns=["Estado de Componentes"])

# Separar las variables de entrada y salida
X = data[["Día", "Mes", "Año", "Hora del día", "Día de la semana", "Watts usados", "Temperatura", "Uso intensivo", "Estado de Componentes_Bueno", "Estado de Componentes_Malo", "Estado de Componentes_Regular", "Porcentaje de uso"]]
y = data["Reporte de fallo"]

# Dividir los datos en un conjunto de entrenamiento y un conjunto de prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear el modelo de Gradient Boosting (XGBoost)
n_estimators_value = 100  # Cambia este valor según tus necesidades
model = xgb.XGBClassifier(objective="binary:logistic", random_state=42, n_estimators=n_estimators_value)

# Entrenar el modelo
model.fit(X_train, y_train)

# Predecir los datos de prueba
y_pred = model.predict(X_test)

# Evaluar el rendimiento del modelo
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

# Imprimir un informe de clasificación
print(classification_report(y_test, y_pred))

Accuracy: 0.50
              precision    recall  f1-score   support

           0       0.50      0.51      0.50     49959
           1       0.50      0.49      0.49     50041

    accuracy                           0.50    100000
   macro avg       0.50      0.50      0.50    100000
weighted avg       0.50      0.50      0.50    100000

